In [13]:
from selenium import webdriver
import time
import pandas as pd
import glob
import os
import shutil
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions, Log
#from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import DataFrame
from pyspark.sql import Window
from datetime import datetime
import psycopg2

# root folder = path 


def horticulture_data_ingesting():
    """
    This function is responsible for scraping excel files from the horticulture website.

    It used a headless firefox browser and loops through the 18 markets. For each market it creates a folder and 
    downloads dynamically all the products within each market which are in the form of excel files.

    Parameters:
    ----------
    None

    Returns:
    -------
    None

    Raises:
    ------
    None


    """
    path = "/Users/hajirufai/internship-project-2207-08/private/tests/extract"

    log = Log()
    log.level = "Trace"
    op = FirefoxOptions()
    # op.add_argument('-headless')
    op.add_argument(log.level)
    op.set_preference("browser.download.folderList", 2)
    op.set_preference("browser.download.manager.showWhenStarting", False)
    op.set_preference("browser.download.dir", f"{path}")
    op.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream")
    gecko = Service(executable_path='/usr/local/bin/geckodriver', log_path = '/opt/airflow/dags/includes/etl/horticulture/geckodriver.log')
    driver = webdriver.Firefox(service = gecko, options=op)
    
    # the file download directory
    d_d = os.path.abspath(f"{path}")

    driver.get('http://webapps.daff.gov.za/amis/amis_price_search.jsp')

    product_size = driver.find_element(By.ID, "cbSearchProduct")

    product_element = Select(product_size)
    product_options = product_element.options
    loop = len(product_options)


    market = driver.find_element(By.ID, "cbSearchMarket")

    market_element = Select(market)
    market_option = market_element.options
    market_size = len(market_option)


    market_name = []
    market_code = []
    market_type = []

   
    f = 9
    while f <= 9:
        # set up a counter to track number of downloaded files
        counter = len(glob.glob(f'{path}/*.xls'))
            
        driver.find_element(By.XPATH, "//select[@id='cbSearchMarket']/option[{}]".format(f)).click()  # selecting market
        driver.find_element(By.NAME, "btnViewMarket").click()  # this selects the view-market-info

        # Optimizing time to catch the object in pop up window
        MAX_TIME = 15  # maximum time in seconds
        start_time = time.time()  # creating a  timestamp
        while True:
            elapsed_time = time.time() - start_time
            if elapsed_time > MAX_TIME:
                raise Exception(f"ERROR!! Maximum time of {MAX_TIME} to load ViewMarketInfo frame")
            else:
                pass
            try:
                # Finds the outer div element containing market info
                outer_div = driver.find_element(By.ID , "popUpDivPDF")

                 # Switches the frame to the object within said div
                driver.switch_to.frame(outer_div.find_element(By.TAG_NAME, 'object'))
                break

            except:
                time.sleep(1)
            
        # # Find the table element within the nested div/object
        table = driver.find_element(By.CSS_SELECTOR, "table:first-child + table")

        # # Find the element within the table and assigns them to variables
        m_n = table.find_element(By.XPATH, "//tr[2]/td[2]")
        m_c = table.find_element(By.XPATH, "//tr[3]/td[2]")
        m_t = table.find_element(By.XPATH, "//tr[4]/td[2]")
        
        # Extracts the text within the elements
        mrkt_name = m_n.text
        mrkt_code = m_c.text
        mrkt_type = m_t.text

        # switches the frame back to the default view and closes the popup window
        driver.switch_to.default_content()
        driver.find_element(By.XPATH, "//img[@title='Click here to close window.']").click()

        market_name.append(mrkt_name)
        market_code.append(mrkt_code)
        market_type.append(mrkt_type)

        for x in range(loop+1):  # loop =187
            if x < 2:
                continue
            try:
                wait1 = WebDriverWait(driver, 50)
                wait1.until(EC.element_to_be_clickable((By.XPATH, "//select[@id='cbSearchMarket']/option[{}]".format(f) ))).click()
            except:
                driver.refresh()
                wait1 = WebDriverWait(driver, 50)
                wait1.until(EC.element_to_be_clickable((By.XPATH, "//select[@id='cbSearchMarket']/option[{}]".format(f) ))).click()
            #driver.find_element(By.XPATH, "//select[@id='cbSearchMarket']/option[{}]".format(f)).click()  # selecting market
            driver.find_element(By.XPATH, "//select[@id='cbPeriod']/option[7]").click()  # selecting period
            driver.find_element(By.XPATH, "//select[@id='cbSearchProduct']/option[{}]".format(x)).click()  # selecting product
            try:    
                # OPTIMIZING TIME
                driver.find_element(By.NAME, "btnDBSearch").click()  # This clicks the view-prices 
                # wait for the EXPORT button to be clickable and then click it
                wait = WebDriverWait(driver, 200)  # max-wait = 200s
                export = wait.until(EC.element_to_be_clickable((By.NAME,"btnExport") ))
            except:
                continue

            # Check if "No price available!" text exists in the page source
            if "NO PRICES AVAILABLE!" in driver.page_source:
                #print("NO PRICES AVAILABLE")
                continue
            else:
                pass

            export.click()  # download
            
            # optimize time to check if a file has been downloaded
            MAX_TIME = 100 
            start_time = time.time() 
            while True:
                elapsed_time = time.time() - start_time
                if elapsed_time > MAX_TIME:
                    # raise Exception(f"ERROR!! Maximum time of {MAX_TIME} to download the file")
                    #print(f"ERROR!! Maximum time of {MAX_TIME} to download the file of {mrkt_name}")
                    break
                else:
                  pass

                # check if a file has been downloaded
                if len(glob.glob(f'{path}/*.xls')) > counter:
                    counter += 1
                    #print("Downloaded", counter)
                    break
                else:
                    # print(f"sleep since {len(glob.glob(f'{path}/*.xls'))} = {counter}")
                    time.sleep(1)
            
           

            
        # print("Moving Downloaded files to their market_code folder")
        source_dir = path
        dest_dir = f'{path}/{mrkt_name}_{mrkt_code}_{mrkt_type}'.strip()
        # create the destination directory if it doesn't exist
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)

        # get a list of all files in the source directory
        file_list = os.listdir(source_dir)

        # loop through the file list and move files that end with ".xls" to the destination directory
        for filename in file_list:
            if filename.endswith(".xls"):
                src_path = os.path.join(source_dir, filename)  # get the source file path
                dest_path = os.path.join(dest_dir, filename)  # get the destination file path
                shutil.move(src_path, dest_path)  # move the file to the destination directory
        f += 1

    
    driver.close()

# 




In [ ]:

horticulture_data_ingesting()
#     preprocessing()
#     processing()
#     # removal_of_folder_containing_excel_files_and_pre_processed_csv()




    








